# GEORREFERENCIAMENTO OVP-DH
## Jupyter Notebook compartilhado

In [1]:
import numpy as np
import pandas as pd
import folium
from folium import plugins
from folium.plugins import MarkerCluster
import json

### Abrindo dataset

In [2]:
#abrindo o arquivo csv e eliminando a coluna sem dados
data = pd.read_csv('../data/georef.csv').drop(columns=['Unnamed: 0'])
#lendo as cinco primeiras linhas
data.head()

,id_ocorrencia,conduta,natureza_lesao,tipo_crime,uf,municipio,endereco,logradouro_sede,geolocalizacao,latitude,longitude
0,2,Operação policial - Repressão a manifestação,Crimes de abuso de autoridade (Lei 4.898/1965),Impedir ou cercear o direito de reunião,RIO DE JANEIRO,Rio de Janeiro,Câmara Municipal,Câmara Municipal,0101000020E6100000E013EB54F91546C0DC847B65DE8A...,-22.542456,-44.171672
1,2,Operação policial - Repressão a manifestação,Crimes de abuso de autoridade (Lei 4.898/1965),Ameaçar a integridade física do indivíduo,RIO DE JANEIRO,Rio de Janeiro,Câmara Municipal,Câmara Municipal,0101000020E6100000E013EB54F91546C0DC847B65DE8A...,-22.542456,-44.171672
2,4,Operação policial - Repressão a manifestação,Crimes de lesão corporal (CP),Lesão corporal grave,DISTRITO FEDERAL,Brasília,Esplanada dos Ministérios,Esplanada dos Ministérios,0101000020E6100000EC53E98255EF47C0B71F95078E98...,-15.797959,-47.869797
3,4,Operação policial - Repressão a manifestação,Crimes de abuso de autoridade (Lei 4.898/1965),Impedir ou cercear o direito de reunião,DISTRITO FEDERAL,Brasília,Esplanada dos Ministérios,Esplanada dos Ministérios,0101000020E6100000EC53E98255EF47C0B71F95078E98...,-15.797959,-47.869797
4,4,Operação policial - Repressão a manifestação,Crimes de abuso de autoridade (Lei 4.898/1965),Ameaçar a integridade física do indivíduo,DISTRITO FEDERAL,Brasília,Esplanada dos Ministérios,Esplanada dos Ministérios,0101000020E6100000EC53E98255EF47C0B71F95078E98...,-15.797959,-47.869797


## Construindo o mapa 

### Definindo 'natureza_lesao' como objetivo de pesquisa

In [3]:
data['natureza_lesao'].unique()

array(['Crimes de abuso de autoridade (Lei 4.898/1965)',
       'Crimes de lesão corporal (CP)', 'Crimes contra a honra (CP)',
       'Crimes contra a liberdade pessoal (CP)',
       'Crimes contra a liberdade sexual (CP)',
       'Crimes contra a vida (CP)',
       'Crimes contra a administração da justiça (CP)',
       'Crimes de tortura (Lei 9.455/1997)',
       'Crimes contra a administração em geral (CP)',
       'Crimes contra a paz pública (CP)',
       'Crimes de periclitação da vida e da saúde (CP)',
       'Crimes de dano (CP)', 'Crimes de perigo comum (CP)',
       'Abuso de autoridade do ECA (Lei 8.069/1990)',
       'Crime de extorsão (CP)', 'Crimes contra a fé pública (CP)',
       'Convenção contra o crime de genocídio'], dtype=object)

In [4]:
# Pegando a média da latitudes e longitudes das ocorrências
latmean = data['latitude'].mean()
lonmean = data['longitude'].mean()

# Criando o mapa
mapa_total = folium.Map(location=[latmean,lonmean], 
                         zoom_start=8, 
                         tiles='OpenStreetMap',
                         control_scale=True)

# Minimapa
minimap = plugins.MiniMap(toggle_display=True)
mapa_total.add_child(minimap)

# Plugins - fullscreen
plugins.Fullscreen(position='topleft').add_to(mapa_total);

### Definindo as cores

In [5]:
dic_nl = {'Crimes de abuso de autoridade (Lei 4.898/1965)':'pink',
          'Crimes de lesão corporal (CP)':'darkred',
          'Crimes contra a honra (CP)':'green',
          'Crimes contra a liberdade pessoal (CP)':'white',
          'Crimes contra a liberdade sexual (CP)':'purple',
          'Crimes contra a vida (CP)':'red',
          'Crimes contra a administração da justiça (CP)':'cadetblue',
          'Crimes de tortura (Lei 9.455/1997)':'black',
          'Crimes contra a administração em geral (CP)':'lightgreen',
          'Crimes contra a paz pública (CP)':'darkgreen',
          'Crimes de periclitação da vida e da saúde (CP)':'darkpurple',
          'Crimes de dano (CP)':'lightred',
          'Crimes de perigo comum (CP)':'orange',
          'Abuso de autoridade do ECA (Lei 8.069/1990)':'darkblue',
          'Crime de extorsão (CP)':'gray',
          'Crimes contra a fé pública (CP)':'blue',
          'Convenção contra o crime de genocídio':'beige'}

### Adicionando o Choropleth

In [6]:
#denominacoes no arquivo json
municipios_geo = r'../data/geojs_estado_sp.json'

#abrir o arquivo na classe dicionario
with open(municipios_geo, encoding='latin-1') as municipios:
    municipios_dict = json.load(municipios)
    
#iterando sobre o dicionario para obter os nomes dos municipios
nomes_json = []
for index in range(len(municipios_dict['features'])):
    nomes_json.append(municipios_dict['features'][index]['properties']['name'])

In [7]:
#transformacao do dataset em tabela para intensidade das camadas
municipios_incidentes = data[data['uf'] == 'SÃO PAULO']['municipio'].value_counts()

In [8]:
#apenas Santa Barbara D'Oeste esta com o nome diferente
#transposicao dos nomes no json para a tabela gerada
municipios_incidentes.rename({"Santa Bárbara D'Oeste":"Santa Bárbara d'Oeste"}, inplace=True)

In [9]:
#selecao dos municios paulistas para aplicar a intensidade encontrada
incidencia_df = pd.DataFrame(municipios_incidentes.reset_index())
incidencia_df.rename({'index':'municipio',
                      'municipio':'incidencia'},
                     axis=1, 
                     inplace = True)

In [10]:
# json completo tem coordenadas demais e mais de 600 municipios
# ha 71 municipios afetados
# selecionar elementos do json que estao na lista de municipios afetados
params_afetados = [x for x in municipios_dict['features'] if x['properties']['name'] in incidencia_df.municipio.to_list()]

# incluir parametros em dicionario a ser formatado
params_dict = {"type": "FeatureCollection", "features": params_afetados}

# criar json, em utf-8 (originalmente latin-1), com base na lista reduzida
geo_data = json.dumps(params_dict)

In [11]:
#criando o mapa do choropleth
folium.Choropleth(
    name='Mapa choropleth',
    geo_data=geo_data,
    data=incidencia_df,
    columns=['municipio', 'incidencia'],
    key_on='feature.properties.name',
    fill_color='OrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='incidentes por municipio',
    smooth_factor=0).add_to(mapa_total);

### Plotando sub-grupos em camadas

In [12]:
#adicionando clusterizacao
mc = MarkerCluster(control=False)
mapa_total.add_child(mc)

#criando html para os popups
html_temp = """
    <h5>Conduta: </h5>
    {}
    <h5>Natureza da lesao: </h5>
    {}
    <h5>Logradouro sede: </h5>
    {}
    """

#definindo a funcao para clusterizacao dos marcadores
def grupo_mapa(natureza_lesao):
    group = plugins.FeatureGroupSubGroup(mc, natureza_lesao)
    mapa_total.add_child(group)
    
    for i in data[data['natureza_lesao'] == natureza_lesao].itertuples():
        lesao = i.natureza_lesao
        if type(lesao) != str:
            lesao = "Indefinida"
        conduta = i.conduta
        if type(conduta) != str:
            conduta = "Indefinida"
        logradouro = i.logradouro_sede 
        if type(logradouro) != str:
            logradouro = "Indefinido"
        variaveis = (lesao, conduta, logradouro)
        html = html_temp.format(lesao, conduta, logradouro)
        popup = folium.Popup(html, max_width=280, min_width=250)
        folium.Marker(location=[i.latitude, i.longitude], 
                      popup = popup, 
                      icon = folium.Icon(color = dic_nl[natureza_lesao], 
                                       icon = 'bolt', 
                                       prefix = 'fa')).add_to(group)
        
#iterando a funcao sobre o set
for i in dic_nl.keys():
    grupo_mapa(i)

In [13]:
# adicionando menu
folium.LayerControl(collapsed=False).add_to(mapa_total);

### Visualizando o mapa

In [14]:
mapa_total

### Salvando o mapa

In [17]:
mapa_total.save('../examples/mapa_total.html')